-----------------------------
# <div style="text-align:center;">TP Spark 1 : notes perso</div>
-------------------------------

## Word count avec un RDD
---------------

### Lire un fichier de données non structurées via un RDD

In [1]:
val rdd = sc.textFile("/home/p5hngk/Downloads/GitHub/INF_729---Introduction_au_framework_Hadoop/cours-spark-telecom-master/README.md")

Intitializing Scala interpreter ...

Spark Web UI available at http://137.194.75.81:4041
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1571126695420)
SparkSession available as 'spark'


rdd: org.apache.spark.rdd.RDD[String] = /home/p5hngk/Downloads/GitHub/INF_729---Introduction_au_framework_Hadoop/cours-spark-telecom-master/README.md MapPartitionsRDD[1] at textFile at <console>:25


Afficher les 5 premières lignes du fichier.

In [2]:
rdd.take(5).foreach(println)

# Cours sur Spark

Vous trouverez ici toutes les ressources pour la partie travaux pratiques du cours sur Spark donné à Télécom pour le Mastère Spécialisé Big Data.

Il y a 3 fichiers de TP:


### Word count

Comptons désormais le nombre de mots dans le fichier et affichons les 10 premières lignes.

In [3]:
val wordCount = rdd
  .flatMap(line => line.split(" "))
  .map(word => (word, 1))
  .reduceByKey((i, j) => i + j)

wordCount.take(10).foreach(println)

(à,1)
(pouvoir,1)
(Cours,1)
(pour,3)
(des,1)
(utilisera,1)
(nécessaire,1)
(plus,1)
(deux,2)
(lors,1)


wordCount: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[4] at reduceByKey at <console>:29


In [4]:
rdd.count

res2: Long = 12


In [5]:
rdd.take(5).foreach(println)

# Cours sur Spark

Vous trouverez ici toutes les ressources pour la partie travaux pratiques du cours sur Spark donné à Télécom pour le Mastère Spécialisé Big Data.

Il y a 3 fichiers de TP:


In [6]:
val flattenedRdd = rdd.flatMap(line => line.split(" "))

flattenedRdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[5] at flatMap at <console>:26


In [7]:
flattenedRdd.count

res4: Long = 109


In [8]:
flattenedRdd.take(10).foreach(println)

#
Cours
sur
Spark

Vous
trouverez
ici
toutes
les


### Digression : types des variables

Pour info, vous pouvez obtenir le type d'un objet en tapant simplement le nom de l'objet dans le shell.

In [11]:
rdd

res8: org.apache.spark.rdd.RDD[String] = /home/p5hngk/Downloads/GitHub/INF_729---Introduction_au_framework_Hadoop/cours-spark-telecom-master/README.md MapPartitionsRDD[1] at textFile at <console>:25


In [12]:
flattenedRdd

res9: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[5] at flatMap at <console>:26


In [13]:
val reducedRdd = flattenedRdd.map(word => (word, 1)).reduceByKey((i, j) => i + j)

reducedRdd: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[7] at reduceByKey at <console>:26


In [15]:
reducedRdd.count

res11: Long = 79


In [16]:
reducedRdd.take(10).foreach(println)

(à,1)
(pouvoir,1)
(Cours,1)
(pour,3)
(des,1)
(utilisera,1)
(nécessaire,1)
(plus,1)
(deux,2)
(lors,1)


### Mots les plus fréquents

In [17]:
// Obtenir les mots les plus fréquents
wordCount
  // wordAndCount est un tuple, on accède à son 2e élément, le count, via ._2
  .sortBy(wordAndCount => wordAndCount._2, ascending=false)
  .take(10)
  .foreach(println)

(,4)
(sur,4)
(Spark,4)
(et,4)
(pour,3)
(de,3)
(-,3)
(qui,3)
(fichiers,3)
(deux,2)


In [18]:
wordCount
  .map(wordAndCount => (wordAndCount._1.toLowerCase, wordAndCount._2))
  .reduceByKey((i, j) => i + j)
  .sortBy(wordAndCount => wordAndCount._2, ascending=false)
  .take(10)
  .foreach(println)

(,4)
(sur,4)
(spark,4)
(et,4)
(pour,3)
(de,3)
(-,3)
(qui,3)
(fichiers,3)
(deux,2)


## Word count avec un DataFrame
-------------------------------

Maintenant qu'on a vu les bases de la manipulation d'un RDD, regardons ce que ça donne avec un DataFrame.

Pour rappel, on doit passer par un SparkSession pour pouvoir utiliser des DataFrames (et Datasets). **Dans le spark-shell** un SparkSession est automatiquement créé pour vous et est accessible en tapant `spark.`

In [22]:
val df = spark.read.text("/home/p5hngk/Downloads/GitHub/INF_729---Introduction_au_framework_Hadoop/cours-spark-telecom-master/README.md")

df: org.apache.spark.sql.DataFrame = [value: string]


In [23]:
df.show(5)

+--------------------+
|               value|
+--------------------+
|   # Cours sur Spark|
|                    |
|Vous trouverez ic...|
|                    |
|Il y a 3 fichiers...|
+--------------------+
only showing top 5 rows



In [25]:
// Par défaut, l'affichage est tronqué, pour afficher toute la ligne :
df.show(5, false)

+-------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                            |
+-------------------------------------------------------------------------------------------------------------------------------------------------+
|# Cours sur Spark                                                                                                                                |
|                                                                                                                                                 |
|Vous trouverez ici toutes les ressources pour la partie travaux pratiques du cours sur Spark donné à Télécom pour le Mastère Spécialisé Big Data.|
|                                                                                                               

In [26]:
val wordCountDF = df
  .withColumn("word", split($"value", " "))
  .withColumn("word", explode($"word")) // équivalent du flatten plus haut
  .groupBy("word")
  .count // crée une colonne "count"

wordCountDF: org.apache.spark.sql.DataFrame = [word: string, count: bigint]


In [28]:
wordCountDF 
    .orderBy($"count".desc)
    .show(10)

+---------+-----+
|     word|count|
+---------+-----+
|         |    4|
|      sur|    4|
|       et|    4|
|    Spark|    4|
|        -|    3|
|     pour|    3|
|      qui|    3|
|       de|    3|
| fichiers|    3|
|trouverez|    2|
+---------+-----+
only showing top 10 rows



In [30]:
// Il faut maintenant passer les mots en minuscules et agréger les résultats :

wordCountDF
    .withColumn("word", lower($"word"))
    .groupBy("word")
    .agg(sum($"count") as "count") // agg : le point d'entrée des fonctions d'agrégation
    .orderBy($"count".desc)
    .show(10)

+----------+-----+
|      word|count|
+----------+-----+
|     spark|    4|
|          |    4|
|       sur|    4|
|        et|    4|
|       qui|    3|
|      pour|    3|
|  fichiers|    3|
|         -|    3|
|        de|    3|
|ressources|    2|
+----------+-----+
only showing top 10 rows



In [31]:
// Autre solution équivalente :

wordCountDF
    .withColumn("word", lower($"word"))
    .groupBy("word")
    .count

wordCountDF
    .orderBy($"count".desc)
    .show(10)

+----------+-----+
|      word|count|
+----------+-----+
|          |    4|
|       sur|    4|
|     Spark|    4|
|        et|    4|
|      pour|    3|
|       qui|    3|
|        de|    3|
|         -|    3|
|  fichiers|    3|
|ressources|    2|
+----------+-----+
only showing top 10 rows



## Bonus : Persistance
----------------------
*Kill du noyau puis relance d'un nouveau.*

In [32]:
val rdd = sc.textFile("/home/p5hngk/Downloads/GitHub/INF_729---Introduction_au_framework_Hadoop/cours-spark-telecom-master/README.md").flatMap(line => line.split(" "))

rdd.map(word => (word, 1)).reduceByKey((i, j) => i + j).collect.foreach(println)

(à,1)
(pouvoir,1)
(Cours,1)
(pour,3)
(des,1)
(utilisera,1)
(nécessaire,1)
(plus,1)
(deux,2)
(lors,1)
(Big,1)
(Mastère,1)
(pratiques,1)
(les,2)
(ce,1)
(installer,1)
(TP:,1)
(,4)
(documentation,1)
(projet,,1)
(donné,1)
(Scala.,1)
(tout,1)
(cours,1)
(qu'on,1)
(Data.,1)
(Ce,1)
(trouverez,2)
(sont,1)
(sa,1)
(ressources,2)
(rassemble,1)
(mais,1)
(autres,1)
(également,2)
(:,1)
(lire,1)
(séances,1)
(toutes,1)
(Spécialisé,1)
(TP.,1)
(sur,4)
(est,2)
([TP_3_machine_learning_avec_spark.md](TP_3_machine_learning_avec_spark.md),1)
(fichiers.,1)
(la,1)
(Vous,2)
(ici,1)
(Spark,4)
(ces,1)
(que,1)
(créer,1)
(Télécom,1)
(généralités,1)
([spark_notes.md](spark_notes.md),1)
(a,1)
(travaux,1)
(de,3)
(y,1)
(le,1)
(recommandé,1)
(et,4)
(#,1)
([TP_1_spark_shell_et_word_count.md](TP_1_spark_shell_et_word_count.md),1)
(faire,1)
([TP_2_projet_et_pre_processings.md](TP_2_projet_et_pre_processings.md),1)
(3,1)
(décrit,1)
(un,1)
(du,1)
(-,3)
(partie,1)
(quelques,1)
(tourner,1)
([setup.md](setup.md),1)
(Il,2)
(qui,3)

rdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[85] at flatMap at <console>:27


Allez ensuite dans l'UI de spark, à l'adresse suivante : 
http://localhost:4040/jobs/


**<span style="color: red">ATTENTION</span>**, pour pouvoir accéder à l'adresse ci-dessus, il faut avoir lancé un `spark-shell` dans un terminal, avec le job de la cellule ci-dessus.

Si l'on relance la même commande et qu'on regarde les mêmes visualisations et métriques, on obtiendra à peu de chose près les mêmes résultats. Cela signifie **qu'à chaque fois** on lit le fichier texte *README* et on effectue la partie `.flatMap(line => line.split(" "))`. Ici c'est très rapide car le fichier est petit et sauvegardé sur notre machine. Mais s'il s'agissait d'un ou plusieurs fichiers de plusieurs giga stockés dans le cloud ? La lecture prendrait beaucoup plus de temps, tout comme le `flatMap`. S'il s'agissait d'un dataset d'entraînement pour un modèle de machine learning sur lequel on doit fait plusieurs passes pour optimiser les hyperparamètres ? Ça serait lent et long.

Pour gérer ce problème, Spark donne la possibilité de **persister le RDD/DataFrame** (que ce soit en mémoire, sur disque, ou un mix des deux) i.e. de sauvegarder l'état actuel du RDD/DataFrame pour ne pas avoir à le recomputer à chaque fois :

In [33]:
rdd.persist
rdd.count // force l'exécution et en particulier la persistance

res26: Long = 109


In [34]:
rdd.map(word => (word, 1)).reduceByKey((i, j) => i + j).collect.foreach(println)

(à,1)
(pouvoir,1)
(Cours,1)
(pour,3)
(des,1)
(utilisera,1)
(nécessaire,1)
(plus,1)
(deux,2)
(lors,1)
(Big,1)
(Mastère,1)
(pratiques,1)
(les,2)
(ce,1)
(installer,1)
(TP:,1)
(,4)
(documentation,1)
(projet,,1)
(donné,1)
(Scala.,1)
(tout,1)
(cours,1)
(qu'on,1)
(Data.,1)
(Ce,1)
(trouverez,2)
(sont,1)
(sa,1)
(ressources,2)
(rassemble,1)
(mais,1)
(autres,1)
(également,2)
(:,1)
(lire,1)
(séances,1)
(toutes,1)
(Spécialisé,1)
(TP.,1)
(sur,4)
(est,2)
([TP_3_machine_learning_avec_spark.md](TP_3_machine_learning_avec_spark.md),1)
(fichiers.,1)
(la,1)
(Vous,2)
(ici,1)
(Spark,4)
(ces,1)
(que,1)
(créer,1)
(Télécom,1)
(généralités,1)
([spark_notes.md](spark_notes.md),1)
(a,1)
(travaux,1)
(de,3)
(y,1)
(le,1)
(recommandé,1)
(et,4)
(#,1)
([TP_1_spark_shell_et_word_count.md](TP_1_spark_shell_et_word_count.md),1)
(faire,1)
([TP_2_projet_et_pre_processings.md](TP_2_projet_et_pre_processings.md),1)
(3,1)
(décrit,1)
(un,1)
(du,1)
(-,3)
(partie,1)
(quelques,1)
(tourner,1)
([setup.md](setup.md),1)
(Il,2)
(qui,3)

Le job a mis cette fois beaucoup moins de temps car la base de données est en mode persistante. Le premier job avait mis **1,0 seconde** à s'éxécuter, alors que le second n'a mis que **0,1 seconde**.


Persister un RDD/DataFrame prend évidemment de la place en mémoire/disque. Quand on n'en a plus besoin, il faut l'*unpersister* :

In [35]:
rdd.unpersist()

res28: rdd.type = MapPartitionsRDD[85] at flatMap at <console>:27
